In [1]:
import torch
from PIL import Image
from torchvision import transforms

In [5]:
## ----------------------------- 确定超参数的值 ----------------------------- ##
num_steps = 100
 
# 制定每一步的beta
betas = torch.linspace(-6, 6, num_steps)
betas = torch.sigmoid(betas) * (0.5e-2 - 1e-5) + 1e-5
alphas = 1 - betas
alphas_prod = torch.cumprod(alphas, 0)
 
# 定义调整noise的常量
alphas_bar_sqrt = torch.sqrt(alphas_prod)
one_minus_alphas_bar_sqrt = torch.sqrt(1 - alphas_prod)

 
## ----------------------------- 确定扩散前向过程任意时刻的采样值 x[t]： x[0] + t --> x[t] ----------------------------- ##
def q_x(x_0, t):
    """
    x[0] + t --> x[t]
    :param x_0:初始数据
    :param t:任意时刻
    """
    noise = torch.randn_like(x_0)
    # 取出在某个时刻t所对应的alphas_t、alphas_1_m_t的值
    alphas_t = alphas_bar_sqrt[t]
    alphas_1_m_t = one_minus_alphas_bar_sqrt[t]
    x_t = alphas_t * x_0 + alphas_1_m_t * noise
    return x_t
 
 


In [6]:
transform=transforms.Compose([
                transforms.Resize((128,128)),
                # transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]), #https://blog.csdn.net/zylooooooooong/article/details/122805833
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
img_path=["/home/yang/sda/github/fuzzydiffusion/doc/multipath/animal.jpg", 
          "/home/yang/sda/github/fuzzydiffusion/doc/multipath/human.jpg",
          "/home/yang/sda/github/fuzzydiffusion/doc/multipath/landscape.jpg"]
img_name=["animal", "human", "landscape"]
batch_size = 3
x_start = torch.rand(batch_size,3,128,128)
for i in range(batch_size):
    path = img_path[i]
    print(path)
    image = Image.open(path)
    img = image.copy()
    image.close()
    img = transform(img)
    x_start[i]=img
# 给x_start加上在timestep 65 的时候噪声
timestep=[10,20,30,40,50,60, 70, 80, 90, 99]
for t in timestep:
    x_noisy = q_x(x_start, t)
    for i in range(batch_size):
        img = x_noisy[i].permute(1, 2, 0).numpy()
        img = (img * 0.5 + 0.5) * 255
        img = img.astype('uint8')
        img = Image.fromarray(img)
        img.save(f"/home/yang/sda/github/fuzzydiffusion/doc/multipath/output/noisy_image_{img_name[i]}_{i}_{t}.png")
noise = torch.torch.rand(3,128,128)
img = noise.permute(1, 2, 0).numpy()
img = (img * 0.5 + 0.5) * 255
img = img.astype('uint8')
img = Image.fromarray(img)
img.save(f"/home/yang/sda/github/fuzzydiffusion/doc/multipath/output/noisy_image.png")


/home/yang/sda/github/fuzzydiffusion/doc/multipath/animal.jpg
/home/yang/sda/github/fuzzydiffusion/doc/multipath/human.jpg
/home/yang/sda/github/fuzzydiffusion/doc/multipath/landscape.jpg
